# Feature Engineering Testing
In this part, we will take a look at our variables and determine which feature engineering methods prep our data in a way that is easier for our models to interpret.

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [10]:
red_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
white_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv"

red_df = pd.read_csv(red_url, sep=";")
white_df = pd.read_csv(white_url, sep=";")
red_df["is_red"] = 1
white_df["is_red"] = 0
df_raw = pd.concat([red_df, white_df])
df_raw.columns = [x.replace(" ", "_") for x in df_raw.columns]

In [6]:
from sklearn.model_selection import train_test_split

df_train_raw, df_test_test = train_test_split(df_raw, test_size=0.2, stratify=df_raw["quality"])

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [ ]:
def get_auc(df, seed, target="quality", split=0.4):

    X = df.drop(target, axis=1)
    y = pd.Categorical(df[target], ordered=True)
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=split, random_state=seed, stratify=y)
    
    phat = LogisticRegression(max_iter=5000, solver="saga")\
        .fit(X_train, y_train)\
        .predict_proba(X_val)
    return roc_auc_score(y_val, phat, multi_class="ovo") 

In [ ]:
auc_list = []
for seed in range(100):
    auc = get_auc(df_train, seed)
    auc_list.append(auc)
    print(auc)


0.5805827034020241


/Users/josephdespres/my_env/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.6295859960131898


/Users/josephdespres/my_env/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.6083587118455158


...

/Users/josephdespres/my_env/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.659954409082519
0.6044171138722705
0.5786813041617083


/Users/josephdespres/my_env/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.6382417245894321


KeyboardInterrupt: 